In [1]:
import json
import requests
from collections.abc import Callable
from typing import Annotated as A, Literal as L

import openai

from annotated_docs.json_schema import as_json_schema

In [2]:
import faiss
import json
# Load the FAISS index
index = faiss.read_index("paintings_index.faiss")

# Load the JSON metadata
with open("metdata.json", "r") as file:
    data = json.load(file)

paintings = data["paintings"]

In [4]:
import os

client = openai.OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [27]:
import os 
client = openai.OpenAI(
    api_key = os.getenv("OPENAI_API_KEY")
)

In [ ]:
class StopException(Exception):
    """
    Stop Execution by raising this exception (Signal that the task is Finished).
    """


def finish(answer: A[str, "Answer to the user's question."]) -> None:
    """Answer the user's question, and finish the conversation."""
    raise StopException(answer)


def search_paintings(query: A[str, "User's question"], top_k: A[int, "Number of results to give"] = 3) -> str:
    """
    Searches the FAISS index for the most relevant paintings based on a user query.

    Args:
        query (str): The user's input query.
        top_k (int): Number of top results to return.
    
    Returns:
        list: List of dictionaries containing painting metadata.
    """
    
    query_embedding = model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, top_k)  # FAISS search
    results = [paintings[i] for i in indices[0]]  # Retrieve painting metadata
    return str(results) 


def search_related_paintings(painting: A[str, "Painting Name"]) -> str:
    """
    Searches for paintings related to the last painting in the conversation.

    Returns:
        list: List of dictionaries containing painting metadata.
    """
    # Search for relevant paintings
    result = search_paintings(painting, top_k=1)
    result = eval(result)
    # Construct a conversational prompt
    related_painting_info = "\n".join(
    [f"Title: {p['painting']}\Artist: {p['artist']}\Why it's related: {p['reason']}" for p in result[0]['related_paintings']]
    )

    return related_painting_info

# function to take user input and return the response

def get_user_input() -> str:
    """
    Get the user's input from the console.
    """
    return input("You: ")



In [33]:
# All functions that can be called by the LLM Agent
name_to_function_map: dict[str, Callable] = {
    search_paintings.__name__: search_paintings,
    search_related_paintings.__name__: search_related_paintings,
    get_user_input.__name__: get_user_input,
    finish.__name__: finish,
}

# JSON Schemas for all functions
function_schemas = [
    {"function": as_json_schema(func), "type": "function"}
    for func in name_to_function_map.values()
]

# Print the JSON Schemas
for schema in function_schemas:
    print(json.dumps(schema, indent=2))

{
  "function": {
    "name": "search_paintings",
    "description": "Searches the FAISS index for the most relevant paintings based on a user query.\n\nArgs:\n    query (str): The user's input query.\n    top_k (int): Number of top results to return.\n\nReturns:\n    list: List of dictionaries containing painting metadata.",
    "parameters": {
      "properties": {
        "query": {
          "type": "string"
        },
        "top_k": {
          "default": 3,
          "type": "integer"
        }
      },
      "required": [
        "query"
      ],
      "type": "object"
    }
  },
  "type": "function"
}
{
  "function": {
    "name": "search_related_paintings",
    "description": "Searches for paintings related to the last painting in the conversation.\n\nReturns:\n    list: List of dictionaries containing painting metadata.",
    "parameters": {
      "properties": {
        "painting": {
          "type": "string"
        }
      },
      "required": [
        "painting"
     

In [35]:
query = "What painting is the one with three girls in red dresses peeling potatoes?"

In [36]:
QUESTION_PROMPT = f"""
    User asked: {query}
    Respond to the user in a conversational tone and provide a brief history of the painting. Then tell them about related paintings. Then ask the user if they have any questions about any of the related paintings or if they would like to know where one of them is located in the museum.
    """

In [43]:
# Initial "chat" messages
messages = [
    {
        "role": "system",
        "content": "You are a helpful museum assistant who can answer multistep questions by sequentially calling functions. Follow a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function). Reason step by step which actions to take to get to the answer. Only call functions with arguments coming verbatim from the user or the output of other functions. Only ask for user input after presenting some information",
    },
    {
        "role": "user",
        "content": QUESTION_PROMPT,
    },
]


def run(messages: list[dict]) -> list[dict]:
    """
    Run the ReAct loop with OpenAI Function Calling.
    """
    # Run in loop
    max_iterations = 20
    for i in range(max_iterations):
        # Send list of messages to get next response
        response = client.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=messages,
            tools=function_schemas,
            tool_choice="auto",
        )
        response_message = response.choices[0].message
        messages.append(response_message)  # Extend conversation with assistant's reply
        # Check if GPT wanted to call a function
        tool_calls = response_message.tool_calls
        if tool_calls:
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                # Validate function name
                if function_name not in name_to_function_map:
                    print(f"Invalid function name: {function_name}")
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"Invalid function name: {function_name!r}",
                        }
                    )
                    continue
                # Get the function to call
                function_to_call: Callable = name_to_function_map[function_name]
                # Try getting the function arguments
                try:
                    function_args_dict = json.loads(tool_call.function.arguments)
                except json.JSONDecodeError as exc:
                    # JSON decoding failed
                    print(f"Error decoding function arguments: {exc}")
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"Error decoding function call `{function_name}` arguments {tool_call.function.arguments!r}! Error: {exc!s}",
                        }
                    )
                    continue
                # Call the selected function with generated arguments
                try:
                    print(
                        f"Calling function {function_name} with args: {json.dumps(function_args_dict)}"
                    )
                    function_response = function_to_call(**function_args_dict)
                    print(f"Function response: {function_response}")
                    # Extend conversation with function response
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )
                except StopException as exc:
                    # Agent wants to stop the conversation (Expected)
                    print(f"Finish task with message: '{exc!s}'")
                    return messages
                except Exception as exc:
                    # Unexpected error calling function
                    print(
                        f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}"
                    )
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}!",
                        }
                    )
                    continue
    return messages


messages = run(messages)

Calling function search_paintings with args: {"query": "three girls in red dresses peeling potatoes"}
Function response: [{'id': '11', 'title': 'The Three Sisters', 'artist': 'Leon Frederic', 'year': '1896', 'description': 'In the 1890s Frederic’s paintings of impoverished workers and peasants in his native Belgium were celebrated for their forthrightness and arresting intensity. Here, the humdrum activity of peeling potatoes is vivified by the girls’ bright red dresses and gleaming red-gold and blond hair. Their downcast eyes and serene expressions recall representations of the young Virgin Mary in sixteenth-century Flemish art, which Frederic greatly admired. Nothing is known of the sitters beyond the painting’s title, which identifies them as sisters; the two eldest are so uncannily alike that they appear to be twins.', 'location': {'room': '827', 'description': 'The painting is located on the south west corner of the gallery. You will see a painting of three girls peeling potatoes 

In [38]:
for message in messages:
    if not isinstance(message, dict):
        message = message.model_dump()  # Pydantic model
    print(json.dumps(message, indent=2))

{
  "role": "system",
  "content": "You are a helpful museum assistant who can answer multistep questions by sequentially calling functions. Follow a pattern of THOUGHT (reason step-by-step about which function to call next), ACTION (call a function to as a next step towards the final answer), OBSERVATION (output of the function). Reason step by step which actions to take to get to the answer. Only call functions with arguments coming verbatim from the user or the output of other functions. Only ask for user input after presenting some information"
}
{
  "role": "user",
  "content": "\n    User asked: What painting is the one with three girls in red dresses peeling potatoes?\n    Respond to the user in a conversational tone and provide a brief history of the painting. Then tell them about related paintings. Then ask the user if they have any questions about any of the related paintings or if they would like to know where one of them is located in the museum.\n    "
}
{
  "content": nul